In [11]:
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import glob

In [12]:
# Globals

filepath_glob_expression = "../data/raw/domestic_holdings/*.txt"
parquet_filepath = "../data/domestic_holdings.parq"
files = glob.glob(filepath_glob_expression)


In [13]:
# TODO Look at why *202012.txt doesn't contain data
#      Deleted for now

all_shapes = []
all_columns = []
for file in files:
    tmp = pd.read_csv(file, nrows=10, sep=',')
    all_shapes.append(tmp.shape)
    all_columns.append(tmp.columns)
assert all([x == all_shapes[0] for x in all_shapes])
assert all([all(x) == all(all_columns[0]) for x in all_columns])

In [14]:
# TODO: Map Headers To English
# TODO: Enforce Expected Data Types

data = dd.read_csv(filepath_glob_expression, sep=";", dtype={'FFM_6010800': 'float64'})

In [15]:
data.head()

,Run Fondo,Nombre Fondo,FFM_6010100,FFM_6010211,FFM_6010212,FFM_6010300,FFM_6010400,FFM_6010500,FFM_6010600,FFM_6010700,...,FFM_REL_6011111,FFM_6011112,FFM_6011113,FFM_6011114,FFM_6011200,FFM_6011300,FFM_6011400,FFM_6011511,FFM_6011512,FFM_6011513
0,8001,CAPITALISA-ACC.,AESGENER,94272000,9,CL,ACC,NaN,1,N-2,...,329.71,3,0,NaN,98254,$$,CL,0.004,0.003,4.381
1,8001,CAPITALISA-ACC.,AGUAS-A,61808000,5,CL,ACC,NaN,1,N-1,...,361.51,3,0,NaN,129279,$$,CL,0.006,0.008,5.764
2,8001,CAPITALISA-ACC.,BCI,97006000,6,CL,ACC,NaN,1,N-1,...,26384.26,3,0,NaN,62504,$$,CL,0.002,0.000,2.787
3,8001,CAPITALISA-ACC.,BSANTANDER,97036000,K,CL,ACC,NaN,1,N-1,...,30.15,3,0,NaN,156188,$$,CL,0.003,0.001,6.964
4,8001,CAPITALISA-ACC.,BUPACL,76005001,6,CL,ACC,NaN,1,N-3,...,507.00,3,0,NaN,58930,$$,CL,0.018,0.012,2.628


In [16]:

data.columns

Index(['Run Fondo', 'Nombre Fondo', 'FFM_6010100', 'FFM_6010211',
       'FFM_6010212', 'FFM_6010300', 'FFM_6010400', 'FFM_6010500',
       'FFM_6010600', 'FFM_6010700', 'FFM_6010800', 'FFM_6010900',
       'FFM_6011000', 'FFM_TIR_6011111', 'FFM_PAR_6011111', 'FFM_REL_6011111',
       'FFM_6011112', 'FFM_6011113', 'FFM_6011114', 'FFM_6011200',
       'FFM_6011300', 'FFM_6011400', 'FFM_6011511', 'FFM_6011512',
       'FFM_6011513'],
      dtype='object')

In [17]:
# TODO: Fill in actual projection

with ProgressBar():
    cols = ['Run Fondo', 'Nombre Fondo', 'FFM_6011513']
    sub = data[cols]
    price_data = sub.compute()

[########################################] | 100% Completed |  7.3s


In [18]:
price_data.head()

,Run Fondo,Nombre Fondo,FFM_6011513
0,8001,CAPITALISA-ACC.,4.381
1,8001,CAPITALISA-ACC.,5.764
2,8001,CAPITALISA-ACC.,2.787
3,8001,CAPITALISA-ACC.,6.964
4,8001,CAPITALISA-ACC.,2.628


In [19]:

price_data.shape

(2702890, 3)

In [20]:
price_data.to_parquet(parquet_filepath, compression='gzip')